In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Optional, Any

import numpy as np
import matplotlib.pyplot as plt

# Session 1: introduction to point processes

When studying wireless communications, the performance of the employed communication schemes is often measured as an averaged quantity, such as the average throughput, or the Bit Error Rate (BER). To compute the average BER on a static channel, the noise realisations should be averaged out. Fading could also comes into play, in which case the error probability is averaged on both the noise and channel realisations. Moreover, it is also possible to consider the slow time channel variations, i.e., the shadowing. 

In these three cases (noise, fading, shadowing), the average quantity can either be determined analytically through Probability Density Functions (PDFs) integration, or through numerical simulations. In these latter, the followed approach is to draw samples of the random quantities, evaluate the performance for each sample draw, and then to take the average of the performances. This approach is named a **Monte-Carlo simulation**. 

Up to now, only the random quantities affecting a single point-to-point link have been discussed. Yet, to study the averaged link quality a user faces in the network, it is important to also consider the spatial distribution of a network as a random quantity. This approach, named **stochastic geometry**, can also be performed either analytically, or by means of Monte-Carlo methods.

The aim of this first project session is to give an introduction to such network sptial distribution analysis. 

## 1.A: Average power on a regular grid

First, let us consider a regular hexagonal grid with hexagon sidelength $R$. We are first interested in studying the average power a user receives from its associated Base Station (BS). To that aim, the path loss model is defined as 

$$ L(r) = \frac{L_0}{\left(h^2+r^2\right)^{\frac{\alpha}{2}}},$$

with $r$ the distance from the BS to the user, $h$ the BS height, $L_0$ the path loss at $r=0$ and $\alpha$ the path loss exponent, belonging to $[2,4]$. 

1. Considering a user with a uniform spatial distribution, write the integral expression giving the average power received by the user from its serving BS.
2. This integral is difficult to obtain analytically. Obtain numerically its value for $L_0=1$, $h=1$, $\alpha = 2,3,4$, and $R\in [0,10]$. 
3. Instead of this numerical integration, the problem can be tackled by approximating the hexagonal cell by a circular one, with a radius $\tilde{R}$. Obtain a closed-form expression of the averaged received power in this case. To be fair, Let $\tilde{R}$ be chosen so as to have the same area the hexagonal cell. 

The three above questions are related to the analytical evaluation of the communication quality. This approach can now be compared to the Monte-Carlo framework. 

4. Using the classes and functions defined below, valide the results obtained analytically, both for the hexagonal and circular cell. 

In [ ]:
@dataclass
class Points:
    """
    2D points with x and y coordinates.
    """
    array: np.ndarray
        
    @property
    def x(self) -> np.ndarray:
        return self.array[:, 0]
        
    @property
    def y(self) -> np.ndarray:
        return self.array[:, 1]
    
    def __len__(self):
        return self.array.shape[0]
    
    def draw(self, ax: Optional[plt.Axes] = None, *args, **kwargs) -> Any:
        if ax is None:
            ax = plt.gca()
        return ax.scatter(self.x, self.y, *args, **kwargs)

class Cell(ABC):
    """
    Abstract class for a cell.
    """
    @abstractmethod
    def sample(self, n: int) -> Points:
        """
        Sample uniformly n points within the current cell.
        """
        pass
    
    @abstractmethod
    def draw(self, ax: Optional[plt.Axes] = None, *args, **kwargs):
        """
        Draw the cell structure on the given axes.
        """
        pass
    
@dataclass
class Circle(Cell):
    radius: float = 1.0
    center: np.ndarray = np.array([0., 0.])
        
    def __post_init__(self):
        self.center = np.asarray(self.center).reshape(1, 2)
    
    def sample(self, n: int) -> Points:
        def sample_in_circle(n):
            xy = np.random.rand(n, 2)
            xy = (xy - .5) * (2 * self.radius)
            r = np.linalg.norm(xy, axis=1)
            
            return xy[r <= self.radius, :]
        
        n2 = n << 1
        xy = sample_in_circle(n2)
        
        while xy.shape[0] < n:
            xy = sample_in_circle(n2)
            
        return Points(array=xy[:n, :] + self.center)
    
    def draw(self, ax: Optional[plt.Axes] = None, *args, fill=False, **kwargs):
        if ax is None:
            ax = plt.gca()
        circle = plt.Circle(self.center.flatten(), self.radius, *args, fill=fill, **kwargs)
        return ax.add_patch(circle)
    
@dataclass
class Hexagon(Cell):
    radius: float = 1.0
    center: np.ndarray = np.array([0., 0.])
    rotation: float = 0.0
        
    def __post_init__(self):
        self.center = np.asarray(self.center).reshape(1, 2)
      
    @property
    def corners_centered_array(self) -> np.ndarray:
        zero = 0.
        radi = self.radius
        s3_2 = np.sqrt(3) * .5 * self.radius
        ra_2 = self.radius * .5
        array = np.array([
            [+s3_2, +ra_2],
            [+zero, +radi],
            [-s3_2, +ra_2],
            [-s3_2, -ra_2],
            [+zero, -radi],
            [+s3_2, -ra_2],
        ])
        return array
    
    @property
    def rotation_array(self) -> np.ndarray:
        cos = np.cos(self.rotation)
        sin = np.sin(self.rotation)
        
        return np.array([
            [+cos, -sin],
            [+sin, +cos],
        ])
    
    @property
    def corners_centered(self) -> Points:
        array = self.corners_centered_array
        return Points(array=array)
    
    @property
    def corners_array(self) -> np.ndarray:
        array = self.corners_centered_array
        array = (self.rotation_array @ array.T).T
        return array + self.center
        
    @property
    def corners(self) -> Points:
        array = self.corners_array
        return Points(array=array)
    
    def sample(self, n: int) -> Points:
        def sample_in_hexagon(n):
            """
            We check that points are in hexagon using
            the 'cross product trick'.
            """
            xy = np.random.rand(n, 2)
            xy = (xy - .5) * (2 * self.radius)
            corners = self.corners_centered_array
            
            # Vectors from points (samples) to each corner
            v1 = - xy[:,:,None] + corners.T[None,:,:]
            v2 = - xy[:,:,None] + np.roll(corners, 1, axis=0).T[None,:,:]
            
            cross = np.cross(v1, v2, axis=1)
            
            # Sum is 6 (or -6) if all the signs are the same
            s = np.sum(np.sign(cross), axis=1)
            index = np.abs(s) == 6
            
            return xy[index, :]
        
        n2 = n << 1
        xy = sample_in_hexagon(n2)
        
        while xy.shape[0] < n:
            xy = sample_in_hexagon(n2)
            
        xy = (self.rotation_array @ xy.T).T
            
        return Points(array=xy[:n, :] + self.center)
    
    def draw(self, ax: Optional[plt.Axes] = None, *args, fill=False, **kwargs):
        if ax is None:
            ax = plt.gca()
        hexagon = plt.Polygon(self.corners_array, *args, fill=fill, **kwargs)
        return ax.add_patch(hexagon)
    
def received_power(points: Points, alpha: float = 2.0) -> np.ndarray:
    r = np.linalg.norm(points.array, axis=1)
    return 1 / (1+r**2)**(alpha/2)

In [ ]:
cell = Circle(radius=3)

points = cell.sample(1000)

cell.draw()
points.draw()

In [ ]:
radii = np.linspace(1, 10)
n = 10000

cell_type = Circle

for alpha in [2., 2.5, 3.]:
    powers = []
    stds = []
    for radius in radii:
        cell = cell_type(radius=radius)
        power = received_power(points=cell.sample(n), alpha=alpha)
        powers.append(np.mean(power))
        stds.append(np.std(power))
        
    plt.errorbar(radii, powers, yerr=stds, label=f"alpha = {alpha}")
    
plt.yscale("log")
plt.legend()

In [ ]:
### Your code here

## 1.B Average power on a random grid

In the above, the case of perfectly regular grids has been considered. Yet, this model does not reflect the inherent irregularity of real world networks. Therefore, another framework is the field of **point processes**, which enable to consider a given network topology a a realisation of a random variable. Then, the performances can be averaged out of all the network topologies. 

Many types of point processes exist, but we will consider in this project **Poisson Point Processes** (PPP), as they are known to provide quite tractable results. Such process is defined as follows. 

Let $\lambda$ be the point density, express in [points/unit area], and consider a given surface $\mathcal{S}$ of area $A$. Then, a realisation of the Poisson point process with density $\lambda$ is obtained by
- Drawing the number of points $N$ from a Poisson distribution with parameter $\lambda A$, whose PDF is given by $f(N=n) = \frac{(\lambda A)^n e^{-\lambda A}}{n!}$.
- Picking the position of these points (i.e. $x$ and $y$ coordinates) independently and uniformly at random in $\mathcal{S}$. 

Implement the below function, which return a realisation of a Poisson point process of point density $\lambda$ in the centered square of sidelength $L$. 

In [ ]:
#Implement PPP function

To study the mean downlink power across a network, one should do the following. 
> Given a large region $\mathcal{S}$, a PPP with density $\lambda_{\text{BS}}$ should be generated, representing the BS. A second PPP with density $\lambda_{\text{UE}}$ sould also be generated, being the User Equipments (UEs). Then, for each UE, the closest BS could be determined and the power could be computed. 

In this process, the UEs located near the boundary of $\mathcal{S}$ should not be taken into account as they might suffer from boundary effects.

Nevertheless, such procedure can be simplified when one is interested in a single user or single BS. Indeed, one of the property of a PPP is that a single point can be added to the PPP without modifying the PPP properties. This means that without loss of generality, the unique UE which is considered for the average power can be considered located at $(x,y)=(0,0)$. The Monte-Carlo simulation takes then the following form. 

> Given a large region $\mathcal{S}$, a PPP with density $\lambda_{\text{BS}}$ should be generated, representing the BS. A typical UE is located at $(x,y) = (0,0)$,  and the power coming from the closest BS can be computed.

Taking advantage of the above functions, compute the average power received by a UE, for $L_0=1$, $h=1$, $\alpha = 2,3,4$, and $\lambda \in [0.1,10]$. For the first method, let $\mathcal{S}$ be a centered square of sidelength $100$, and make the average only on UEs inside a square of sidelength $50$ to avoid boundary effects. For the second method, a square of sidelength $30$ is sufficient.
Compare the number of draws which need to be performed in order to have smooth curves with the two methods, and discuss the complexity of both methods. 

In [ ]:
#Implement Monte-Carlo simulation - method 1

In [ ]:
#Implement Monte-Carlo simulation - method 2

The Poisson property of PPP is particularly useful when considering such problem from their analytic counterparts. Yet, such analytic development is not the aim of this project and we will always consider Monte-Carlo simulations to evaluate the performances in the following. 

## 1.C Average exposure on a random grid

The above focuses on the average power received from the BS serving a UE. While this is already very interesting, this does not fully use the power of the framework of point processes. Indeed, a single user and a singly BS are considered, and thus one could avoir generating the whole network to evaluate the performance.  

To investigate the use of point processes, one can consider the exposure a UE gets in a network, i.e. the sum of the power coming from all the BSs. 

Implement a Monte-Carlo method investigating how the exposure evolves with the BS density, for $L_0=1$, $h=1$, $\alpha = 2,3,4$, and $\lambda \in [0.1,10]$. Consider only the second way of doing the Monte-Carlo simulation, i.e., with a centered typical user. You can use a square $\mathcal{S}$ of length $100$ to begin. Among others, what happens when $\alpha = 2$? Consider a square of length $200$, and $300$ to understand this phenomenon. 

In [ ]:
#Implement Monte-Carlo simulation - exposure